**UNDER CONSTRUCTION...**

Forked from:   

[What do Kagglers say about Data Science ?](https://www.kaggle.com/mhajabri/what-do-kagglers-say-about-data-science) -- Chris Crawford

[What affects your compensation?](https://www.kaggle.com/christinampoid/what-affects-your-compensation) -- Christina Boididou

[Analyzing the analyzers](https://www.kaggle.com/crawford/analyzing-the-analyzers) -- Mhamed Jabri

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set_style("whitegrid")
import operator

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
conversion_rates = pd.read_csv('../input/conversionRates.csv', encoding="ISO-8859-1")
free_form = pd.read_csv('../input/freeformResponses.csv', encoding="ISO-8859-1")
data = pd.read_csv('../input/multipleChoiceResponses.csv', encoding="ISO-8859-1")
schema = pd.read_csv('../input/schema.csv', encoding="ISO-8859-1")

In [ ]:
data.head(10)

In [ ]:
# Gender

plt.figure(figsize=(12,8))
genders = data['GenderSelect'].value_counts()  # returns unique values from the series
sns.barplot(y=genders.index, x=genders.values, alpha=0.6)
plt.yticks(range(len(data['GenderSelect'].value_counts().index)), ['Male', 'Female', 'Different identity', 'Non-confirming'])
plt.title("Gender Distribution of the survey participation", fontsize=16)
plt.xlabel("Number of participants", fontsize=16)
plt.ylabel("Gender", fontsize=16)
plt.show();
print('Proportion of female in this survey: {:0.2f}% '.format(100*len(data[data['GenderSelect']=='Female'])/len(data['GenderSelect'].dropna())))
print('Proportion of men in this survey: {:0.2f}% '.format(100*len(data[data['GenderSelect']=='Male'])/len(data['GenderSelect'].dropna())))

In [ ]:
# Age

plt.hist(data['Age'].dropna(), 10, facecolor='blue', alpha=0.6, label='Frequency')
plt.hist(data['Age'].dropna(), 10, cumulative=True, facecolor='green',alpha=0.2, label='Cumulative')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Country

plt.figure(figsize=(12,8))
countries = data['Country'].value_counts().drop("Other").head(10)
sns.barplot(y=countries.index, x=countries.values, alpha=0.6)
plt.title("Country Distribution of the Survey Participants", fontsize=16)
plt.xlabel("Number of participants", fontsize=16)
plt.ylabel("Country", fontsize=16)
plt.show()

In [ ]:
print('{:0.2f}% of the instances are Americans'.format(100*len(data[data['Country']=='United States'])/len(data)))
print('{:0.2f}% of the instances are Indians'.format(100*len(data[data['Country']=='India'])/len(data)))

In [ ]:
# Majors

plt.figure(figsize=(12,8))
majors = data['MajorSelect'].value_counts()
sns.barplot(y=majors.index, x=majors.values, alpha=0.6)
plt.title("Majors of the survey participants", fontsize=16)
plt.xlabel("Number of participants", fontsize=16)
plt.ylabel("Major", fontsize=16)
plt.show()

In [ ]:
# Motivation

# Import WordCloud library
import wordcloud

# Process text data
text = free_form[pd.notnull(free_form["KaggleMotivationFreeForm"])]["KaggleMotivationFreeForm"]

# Generate the wordcloud
word_cloud = wordcloud.WordCloud(height=600, width=800, background_color="white", colormap="Blues", 
                                 relative_scaling=0.2, random_state=0).generate(" ".join(text))

# Show plot
plt.figure(figsize=(12,12))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")

In [ ]:
# Create feature "Python_vs_R"

df_python_r = data[["WorkToolsFrequencyR","WorkToolsFrequencyPython"]].fillna(0)
df_python_r.replace(to_replace=['Rarely','Sometimes','Often','Most of the time'],
                    value=[1,2,3,4], inplace=True)
df_python_r['Python_vs_R'] = [ 'R' if (freq1 >2 and freq1 > freq2) else
                                'Python' if (freq1<freq2 and freq2>2) else
                                'Both' if (freq1==freq2 and freq1 >2) else
                                'None' for (freq1,freq2) in zip(df_python_r["WorkToolsFrequencyR"],df_python_r["WorkToolsFrequencyPython"])]
data['Python_vs_R'] = df_python_r['Python_vs_R']


In [ ]:
# Create filter of demographic features

demographic_features = ['GenderSelect','Country','Age',
                        'FormalEducation','MajorSelect','ParentsEducation',
                        'EmploymentStatus', 'CurrentJobTitleSelect',
                        'DataScienceIdentitySelect','CodeWriter',
                        'CurrentEmployerType','JobFunctionSelect',
                        'SalaryChange','RemoteWork','WorkMLTeamSeatSelect',
                        'Tenure','EmployerIndustry','EmployerSize','Python_vs_R',
                        'CompensationAmount']

# Retrieve relevant columns
data_compensation = data[demographic_features]
data_compensation.head(5)

In [ ]:
# Only look at USA data
data_compensation_usa = data_compensation[data_compensation['Country']=='United States']

# Only keep those filled out the compensation data
data_compensation_usa = data_compensation_usa[data_compensation_usa['CompensationAmount'].notnull()]

# Convert all to 'str' and remove the commas
data_compensation_usa['CompensationAmountClean'] = data_compensation_usa['CompensationAmount'].str.replace(',', '')

# Remove the '-' cells
data_compensation_usa = data_compensation_usa.loc[~data_compensation_usa['CompensationAmountClean'].str.contains('-')]

# Convert to 'float' type
data_compensation_usa['CompensationAmountClean'] = data_compensation_usa['CompensationAmountClean'].astype(float)

# Remove outliers (<$10,000 or >$1000,000 annually)
data_compensation_usa = data_compensation_usa[(data_compensation_usa['CompensationAmountClean']>10000) & 
                                              (data_compensation_usa['CompensationAmountClean']<1000000)]

data_compensation_usa.head(10)

In [ ]:
# Plot the income distribution

plt.subplots(figsize=(15,8))
sns.distplot(data_compensation_usa['CompensationAmountClean'])
plt.title("Income Histogram and Fitted Distribution", size=16)
plt.show()

In [ ]:
# Boxplot of income

plt.figure(figsize=(4,6))
sns.boxplot(y=data_compensation_usa['CompensationAmountClean'], data=data_compensation_usa)
plt.title("Compensation Box Plot", fontsize=16)
plt.ylabel("Compensation", fontsize=16)
plt.show()

In [ ]:
# Violin Plot by Genders

plt.figure(figsize=(10,8))
sns.violinplot(y='CompensationAmountClean', x='GenderSelect', data=data_compensation_usa)
plt.title("Salary Distribution Violin Plot", fontsize=16)
plt.ylabel("Annual Salary", fontsize=16)
plt.xlabel("Gender", fontsize=16)
plt.show()

In [ ]:
# Salary by Education Level

education_levels = data_compensation_usa.groupby('FormalEducation')['CompensationAmountClean'].median().fillna(0).sort_values()[::-1].index
data_education = data_compensation_usa[data_compensation_usa.FormalEducation.isin(education_levels)]

plt.figure(figsize=(10,8))
sns.boxplot(x='CompensationAmountClean', y='FormalEducation', data=data_education, order=education_levels)
plt.title("Salary distribution by education levels", fontsize=16)
plt.xlabel("Annual Salary", fontsize=16)
plt.ylabel("Education level", fontsize=16)
plt.show()

In [ ]:
# Salary by Job Title

job_titles = data_compensation_usa.groupby('CurrentJobTitleSelect')['CompensationAmountClean'].median().fillna(0).sort_values()[::-1].index
data_job_title = data_compensation_usa[data_compensation_usa.CurrentJobTitleSelect.isin(job_titles)]

plt.figure(figsize=(10,8))
sns.boxplot(x='CompensationAmountClean', y='CurrentJobTitleSelect', data=data_job_title, order=job_titles)
plt.title("Salary distribution by job titles", fontsize=16)
plt.xlabel("Annual Salary", fontsize=16)
plt.ylabel("Job titles", fontsize=16)
plt.show()

In [ ]:
# Salary by Language Skills

python_vs_r = data_compensation_usa.groupby('Python_vs_R')['CompensationAmountClean'].median().fillna(0).sort_values()[::-1].index
data_python_vs_r = data_compensation_usa[data_compensation_usa.Python_vs_R.isin(python_vs_r)]

plt.figure(figsize=(10,8))
sns.boxplot(x='CompensationAmountClean', y='Python_vs_R', data=data_python_vs_r, order=python_vs_r)
plt.title("Salary distribution by programming language", fontsize=16)
plt.xlabel("Annual Salary", fontsize=16)
plt.ylabel("Programming Language", fontsize=16)
plt.show()

In [ ]:
# Age vs Salary

plt.figure(figsize=(8,8))
plt.scatter(data_compensation_usa['Age'], data_compensation_usa['CompensationAmountClean'])
plt.title("Salary vs Age", fontsize=16)
plt.xlabel("Age", fontsize=16)
plt.ylabel("Annual Salary", fontsize=16)
plt.show()

In [ ]:
# Regression on Salary

# Import StatsModels library
import statsmodels.formula.api as smf

# Construct formula
linear_model_salary = smf.ols('CompensationAmountClean ~ Age + Python_vs_R \
                              + CurrentJobTitleSelect + FormalEducation + GenderSelect \
                              + MajorSelect + ParentsEducation + EmploymentStatus \
                              + RemoteWork + Tenure + EmployerIndustry + EmployerSize', data=data_compensation_usa).fit()

linear_model_salary.summary()

In [ ]:
# Plot the regression plots

# Import StatsModels library
import statsmodels.api as sm

fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(linear_model_salary, "Age", fig=fig)

In [ ]:
# Categorical List

categorical_features = demographic_features.copy()
categorical_features.remove('CompensationAmount')
categorical_features.remove('Age')
categorical_features.remove('Country')
categorical_features

In [ ]:
# Check out the subset head head

data_compensation_usa[demographic_features].head()

In [ ]:
data_compensation_usa.info()

In [ ]:
# Drop the rows where 'Age' is null

data_filter_age = data_compensation_usa[pd.notnull(data_compensation_usa['Age'])]
data_filter_age.info()

In [ ]:
# Regression on Salary 

# Import scikit-learn
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

y = data_filter_age['CompensationAmountClean']
X = data_filter_age[categorical_features]  # Limit to categorical features first

X.head()

In [ ]:
# Check the shape of data
X.shape

In [ ]:
# Check missing values
X.info()

In [ ]:
# Deal with missing values

X_fill = X.fillna("NA")
X_fill.info()

In [ ]:
# Label encoding

label_enc = LabelEncoder()
X_lab = X_fill.apply(label_enc.fit_transform)
X_lab.head()

In [ ]:
# One Hot Encoding

oh_enc = OneHotEncoder()
oh_enc.fit(X_lab)
X_oh = oh_enc.transform(X_lab).toarray()
X_oh.shape   
# Elisa's comment: It seems like not able to get the feature names

In [ ]:
# Use pandas instead

X_dummy = pd.get_dummies(X_fill)
X_dummy.head()

In [ ]:
X_dummy.shape

In [ ]:
# Combine with Age

X_mix = X_dummy.copy()
X_mix['Age'] = data_compensation_usa['Age']
X_mix.head()

In [ ]:
X_mix.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_mix, y, test_size=0.2, random_state=0)

print("Shape of")
print("Training X: {} ".format(X_train.shape))
print("Training y: {} ".format(y_train.shape))
print("Testing X: {} ".format(X_test.shape))
print("Testing y: {} ".format(y_test.shape))

In [ ]:
X_train.isnull().sum()

In [ ]:
# Retrieve feature names for label texts
feature_names = list(X_train.columns)
print(feature_names)

In [ ]:
# If max depth is 2

dt_regressor_d2 = DecisionTreeRegressor(max_depth=2)
dt_regressor_d2.fit(X_train, y_train)

# Visualize the tree
export_graphviz(dt_regressor_d2, out_file=None, feature_names=feature_names)

In [ ]:
# If max depth is 4

dt_regressor_d4 = DecisionTreeRegressor(max_depth=4)
dt_regressor_d4.fit(X_train, y_train)

# Visualize the tree
export_graphviz(dt_regressor_d4, out_file=None, feature_names=feature_names)

In [ ]:
# Another type of look
# import pydotplus
# from sklearn.externals.six import StringIO
# from IPython.display import Image

# dot_data = StringIO()

# export_graphviz(dt_regressor_d2, out_file=dot_data, 
#                 filled=True, rounded=True, special_characters=True)
# graph_d2 = pydotplus.graph_from_dot_data(dot_data.getvalue())
# Image(graph_d2.create_png)

# ModuleNotFoundError                       Traceback (most recent call last)
# <ipython-input-2-ae95a0217a52> in <module>()
#       1 # Another type of look
# ----> 2 import pydotplus
#       3 from sklearn.externals.six import StringIO
#       4 from IPython.display import Image
#       5 

# ModuleNotFoundError: No module named 'pydotplus'

In [ ]:
# Produce performance score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

dt_fit_d2 = dt_regressor_d2.fit(X_train, y_train)
score_r2_d2 = r2_score(y_test, dt_fit_d2.predict(X_test))
score_accuracy_d2 = dt_fit_d2.score(X_train, y_train)
score_cv_d2 = cross_val_score(dt_fit_d2, X_train, y_train, cv=5)

dt_fit_d4 = dt_regressor_d4.fit(X_train, y_train)
score_r2_d4 = r2_score(y_test, dt_fit_d4.predict(X_test))
score_accuracy_d4 = dt_fit_d4.score(X_train, y_train)
score_cv_d4 = cross_val_score(dt_fit_d4, X_train, y_train, cv=5)

print("R^2 score for depth=2 tree: {:.2f}".format(score_r2_d2))
print("Accuracy score for depth=2 tree: {:.2f}".format(score_accuracy_d2))
print("Cross Validation score for depth=2 tree: {:.2f}".format(score_accuracy_d2))

print("R^2 score for depth=4 tree: {:.2f}".format(score_r2_d4))
print("Accuracy score for depth=4 tree: {:.2f}".format(score_accuracy_d4))
print("Cross Validation score for depth=4 tree: {:.2f}".format(score_accuracy_d4))